In [5]:
import pandas as pd
import lightgbm as lgb
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

print("--- Étape 2 : Entraînement du Modèle de Tarification (Taux) ---")

# --- 1. Chargement des données et des assets du Modèle 1 ---
input_path = "../../data/processed/dataset_clean_no_outliers.parquet"
output_dir = "../../models/"
model_1_path = os.path.join(output_dir, "step1_risk_model.pkl")
model_1_cols_path = os.path.join(output_dir, "step1_risk_model_columns.pkl")

df = pd.read_parquet(input_path)
model_1 = joblib.load(model_1_path)
model_1_columns = joblib.load(model_1_cols_path)
print("✅ Données et assets du Modèle 1 chargés.")

# --- 2. Calcul du Score de Risque ---
# Préparation des données pour le Modèle 1
selected_features_for_risk_model = [
    'term', 'annual_inc', 'dti', 'revol_util', 'revol_bal', 'loan_amnt', 
    'emp_length', 'home_ownership', 'purpose', 'verification_status', 
    'mort_acc', 'pub_rec', 'open_acc', 'total_acc'
]
X_for_risk_prediction = df[selected_features_for_risk_model]
X_for_risk_prediction = pd.get_dummies(X_for_risk_prediction, drop_first=True, dtype=float)
X_for_risk_prediction = X_for_risk_prediction.reindex(columns=model_1_columns, fill_value=0)

# Ajout du score de risque au DataFrame principal
df['risk_score'] = model_1.predict_proba(X_for_risk_prediction)[:, 1]
print("✅ Score de risque calculé.")

# --- 3. Préparation des données pour le Modèle de Tarification ---
# La Cible (Y) est le taux d'intérêt en format décimal (ex: 0.075 pour 7.5%)
y = df['int_rate']

# Les Features (X) sont le score de risque et les infos du prêt, incluant le grade
features_for_rate_model = [
    'risk_score',
    'loan_amnt',
    'annual_inc',
    'dti',
    'term',
    'grade' # On inclut le grade car il est le principal déterminant du taux
]
X = df[features_for_rate_model]
X = pd.get_dummies(X, drop_first=True, dtype=float)

# --- 4. Entraînement du Modèle de Régression ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rate_model = lgb.LGBMRegressor(random_state=42)
print("Entraînement du modèle de tarification...")
rate_model.fit(X_train, y_train)
print("✅ Modèle de tarification entraîné.")

# --- 5. Évaluation ---
y_pred = rate_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"\n--- Performance du Modèle de Tarification ---")
print(f"Score R² : {r2:.4f} (part de la variance expliquée)")
print(f"Erreur Absolue Moyenne (MAE) : {mae*100:.2f} points de pourcentage")

# --- 6. Sauvegarde ---
rate_model_path = os.path.join(output_dir, "step2_rate_model.pkl")
rate_model_cols_path = os.path.join(output_dir, "step2_rate_model_columns.pkl")

joblib.dump(rate_model, rate_model_path)
joblib.dump(X_train.columns.tolist(), rate_model_cols_path)
print(f"\n✅ Modèle de tarification (Étape 2) et ses colonnes sauvegardés.")

--- Étape 2 : Entraînement du Modèle de Tarification (Taux) ---
✅ Données et assets du Modèle 1 chargés.
✅ Score de risque calculé.
Entraînement du modèle de tarification...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000898 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1034
[LightGBM] [Info] Number of data points in the train set: 292971, number of used features: 11
[LightGBM] [Info] Start training from score 12.328853
✅ Modèle de tarification entraîné.

--- Performance du Modèle de Tarification ---
Score R² : 0.5386 (part de la variance expliquée)
Erreur Absolue Moyenne (MAE) : 216.45 points de pourcentage

✅ Modèle de tarification (Étape 2) et ses colonnes sauvegardés.
